# Personality-Based Interview Exploration

**Purpose:**  
This notebook demonstrates how interviewer personality affects question generation,
answer evaluation, strictness, and feedback within the AI Interviewer system.

Using a controlled setup with the same candidate input, this notebook compares the
behavior of four interviewer personalities—Friendly Coach, Startup Hiring Manager,
Professor, and Strict FAANG—to highlight differences in tone, expectations, and
decision-making.

This notebook is supplementary to the main application and focuses specifically on
personality-driven behavior rather than end-to-end interview execution.


In [1]:
import sys
import os

# Add src/ to Python path
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

print("SRC path added:", SRC_PATH)


SRC path added: d:\AI_Interviewer_agent\src


In [2]:
from interviewer.state import CandidateInfo, InterviewState
from interviewer.questions import build_technical_question
from interviewer.graph import judge_answer, apply_personality_rules
from interviewer.evaluation import evaluate_interview
from interviewer.llm import LLM


In [3]:
candidate_answer = (
    "I designed a REST API with authentication and caching. "
    "I used JWT for auth and Redis for caching to reduce latency."
)

question = "How would you design a scalable backend API?"


In [4]:
llm = LLM()

personalities = [
    "Friendly Coach",
    "Startup Hiring Manager",
    "Professor",
    "Strict FAANG"
]

results = {}

for p in personalities:
    label = judge_answer(
        llm=llm,
        question=question,
        answer=candidate_answer,
        personality=p
    )
    results[p] = label

results


{'Friendly Coach': 'correct',
 'Startup Hiring Manager': 'partial',
 'Professor': 'partial',
 'Strict FAANG': 'partial'}

### Observation

For the same technical answer:
- Friendly Coach accepts broad understanding
- Startup Hiring Manager values practical decisions
- Professor expects deeper theory and formal explanation
- Strict FAANG enforces strict correctness and precision

This demonstrates personality-controlled evaluation behavior.


In [5]:
questions = {}

for p in personalities:
    q = build_technical_question(
        llm=llm,
        user_id="demo",
        role="Backend Engineer",
        resume_text="Worked on APIs, databases, caching systems",
        experience="Mid",
        run_nonce="test",
        personality=p
    )
    questions[p] = q["question"]

questions


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
d:\AI_Interviewer_agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


{'Friendly Coach': "You're building a simple API to manage books. It should have the following endpoints: GET /books (list all books), GET /books/:id (get a single book), POST /books (create a new book), PUT /books/:id (update an existing book), and DELETE /books/:id (delete a book). Implement this API in Python using Flask. Consider using a database to store the books, and make sure to handle errors properly.",
 'Startup Hiring Manager': "You're working on a high-traffic e-commerce API that serves product information to millions of users daily. The current API design uses a straightforward GET request to fetch a product by ID. However, this approach leads to a significant number of cache misses, resulting in increased latency and database queries. Describe a caching strategy you'd implement to improve the API's performance and scalability, considering the production constraints of handling high traffic and the need to cache frequently accessed products.",
 'Professor': 'Consider a RES

### Personality-Based Question Generation Results

The output above shows the technical interview questions generated for the same
candidate profile under four different interviewer personalities.

Although the role, experience level, and resume context are identical, each
personality produces a distinct question style:

- **Friendly Coach**  
  Focuses on practical implementation and learning-oriented scenarios.  
  The question emphasizes writing a function, handling session expiration, and
  applying concepts in a guided, supportive manner.

- **Startup Hiring Manager**  
  Emphasizes real-world production challenges such as high traffic, scalability,
  and trade-offs between cost, complexity, and performance.  
  The question reflects startup environments where impact and decision-making
  under constraints are critical.

- **Professor**  
  Prioritizes theoretical depth, formal system design, and correctness.  
  The question requires understanding of distributed systems, sharding,
  consistency guarantees, and architectural reasoning.

- **Strict FAANG**  
  Enforces the highest level of difficulty and precision.  
  The question demands a comprehensive system design covering scalability,
  replication, consistency models, failure handling, and complex transactional
  relationships at large scale.

These results demonstrate that interviewer personality directly influences
question difficulty, focus, and expected depth, validating the personality-driven
question generation logic used in the system.


### Whiteboard Behavior Across Personalities

During the Technical round, all interviewer personalities allow whiteboard usage.
However, the feedback and follow-up behavior differs:

- **Friendly Coach** gives supportive and encouraging feedback
- **Startup Hiring Manager** focuses on system bottlenecks and real-world risks
- **Professor** highlights missing theoretical steps and formal reasoning gaps
- **Strict FAANG** penalizes incorrect or incomplete diagrams strictly

Whiteboard analysis uses:
- **Primary:** OpenRouter vision model for diagram analysis
- **Fallback:** LLaMA 3.2 Vision-Instruct (text-only) when vision analysis is unavailable
